# Knapsack problem

Given a list of items and a knapsack capacity, find the maximum profit that this knapsack can carry. Each item contains a value and a weight, the goal is select items to maximize the combined value (sum of all items values selected) in a way that the selected items weights combined do not overspace the capacity of the knapsack.

There are some approaches to tackle this problem, this notebook uses the following:

- brute force
- dynamic programming
- neural knapsack

## Brute force

it consists in ....

In [1]:
# code brute force
# TODO

## Dynamic programming

This method builds a matrix containing possible knapsack configurations. The last digit in the matrix (last row and last column) holds the maximum profit arrangement, since the algorithm choose the maximum value, between adding the current item or just use repeat the last configuration. 

Space time complexity - O(items*capacity)

In [2]:
# code dynamic programming
capacity = 50
items = [[60, 10], [100, 20], [120, 30]]

def getIndicesItems(values, items):
	indices = []
	i = len(values) - 1
	j = len(values[i]) - 1
	while i > 0:
		if values[i][j] != values[i - 1][j]:
			# Remember, comparing to items array we got an extra [] item
			indices.append(i - 1)
			wItem = items[i - 1][1]
			j -= wItem
		i -= 1
	return indices

def knapsackProblem(items, capacity):
	values = [[0 for _ in range(capacity + 1)] for _ in range(len(items) + 1)]
	for i in range(1, len(values)):
		for j in range(len(values[i])):
			currentItem = items[i - 1] # because we have an [] extra item
			v, w = currentItem
			if w <= j:
				previousValue = values[i - 1][j]
				currentValue = values[i - 1][j - w] + v
				values[i][j] = max(previousValue, currentValue)
			else:
				values[i][j] = values[i - 1][j]
	indices = getIndicesItems(values, items)
	combinedValue = sum([items[i][0] for i in indices])
	return [combinedValue, indices]

knapsackProblem(items, capacity)

[220, [2, 1]]

## Neural knapsack

Code based on the article "Neural Knapsack", by Shayan Hashemi

Supervised learning approach.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import binary_accuracy, binary_crossentropy
import tensorflow.keras.backend as K
import pandas as pd
tf.compat.v1.disable_eager_execution()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
INFO:tensorflow:Disabling eager execution


In [4]:
def brute_force_knapsack(x_weights, x_prices, x_capacity):
    item_count = x_weights.shape[0]
    picks_space = 2 ** item_count
    best_price = -1
    best_picks = np.zeros(item_count)
    for p in range(picks_space):
        picks = [int(c) for c in f"{p:0{item_count}b}"]
        price = np.dot(x_prices, picks)
        weight = np.dot(x_weights, picks)
        if weight <= x_capacity and price > best_price:
            best_price = price
            best_picks = picks
    return best_picks


def create_knapsack(item_count=5):
    x_weights = np.random.randint(1, 45, item_count)
    x_prices = np.random.randint(1, 99, item_count)
    x_capacity = np.random.randint(1, 99)
    y = brute_force_knapsack(x_weights, x_prices, x_capacity)
    return x_weights, x_prices, x_capacity, y
x_weights, x_prices, x_capacity, y = create_knapsack()
print("Weights:")
print(x_weights)
print("Prices:")
print(x_prices)
print("Capacity:")
print(x_capacity)
print("Optimum solution: ")
print(y)
print("\nData normalized to be passed into the neural newtork:\n")
x_weights = x_weights / x_capacity
print("Weights:")
print(x_weights)
x_prices = x_prices / max(x_prices)
print("Prices:")
print(x_prices)
print(y)

Weights:
[17  3 10 32 13]
Prices:
[ 6 95 60 92 16]
Capacity:
82
Optimum solution: 
[1, 1, 1, 1, 1]

Data normalized to be passed into the neural newtork:

Weights:
[0.20731707 0.03658537 0.12195122 0.3902439  0.15853659]
Prices:
[0.06315789 1.         0.63157895 0.96842105 0.16842105]
[1, 1, 1, 1, 1]


In [5]:
samples_training = 9000
samples_testing = 1000

def build_dataset(samples, filename):
  data =  []
  for i in range(samples):
    x_weights, x_prices, x_capacity, y = create_knapsack()
    # normalize data
    x_weights = x_weights / x_capacity
    x_prices = x_prices / max(x_prices)
    row = np.concatenate((x_weights, x_prices, y))
    data.append(row)
  df = pd.DataFrame(data, columns=('w0', 'w1', 'w2', 'w3', 'w4', 'p0', 'p1', 'p2', 'p3', 'p4', 'y0', 'y1', 'y2', 'y3', 'y4'))
  csv_text = df.to_csv(index=False, line_terminator='\n')
  with open(filename, 'w') as writer:
    writer.write(csv_text)

build_dataset(samples_training, 'knapsack_train_dataset.csv')
build_dataset(samples_testing, 'knapsack_test_dataset.csv')

In [6]:
def get_dataset(dataset_csv):
  df = pd.read_csv(dataset_csv)
  x_weights_df = df[['w0', 'w1', 'w2', 'w3', 'w4']]
  x_prices_df = df[['p0', 'p1', 'p2', 'p3', 'p4']]
  y_df = df[['y0', 'y1', 'y2', 'y3', 'y4']]
  # x_weights = tf.convert_to_tensor(x_weights_df)
  x_weights = x_weights_df.to_numpy()
  # x_prices = tf.convert_to_tensor(x_prices_df)
  x_prices = x_prices_df.to_numpy()
  # y = tf.convert_to_tensor(y_df)
  y = y_df.to_numpy()
  return [x_weights, x_prices], y

x_train, y_train = get_dataset('./knapsack_train_dataset.csv')
x_test, y_test = get_dataset('./knapsack_test_dataset.csv')

In [7]:
def get_model(item_count=5):
    input_weights = Input((item_count,))
    input_prices = Input((item_count,))
    inputs_concat = Concatenate()([input_weights, input_prices])
    picks = Dense(35, activation="sigmoid")(inputs_concat)
    picks = Dense(item_count, activation="sigmoid")(picks)
    model = Model(inputs=[input_weights, input_prices], outputs=[picks])
    model.compile(
        'adam',
        tf.keras.losses.binary_crossentropy,
        metrics = [
          tf.keras.metrics.binary_accuracy,
          metric_space_violation(input_weights),
          metric_overprice(input_prices)
          ]
    )
    return model

def get_model_test2():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10),
    tf.keras.layers.Dense(35, activation='sigmoid'),
    tf.keras.layers.Dense(5, activation='sigmoid')
  ])
  return model

In [8]:
def metric_overprice(input_prices):
    def overpricing(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.batch_dot(y_pred, input_prices, 1) - K.batch_dot(y_true, input_prices, 1))
    return overpricing
def metric_space_violation(input_weights):
    def space_violation(y_true, y_pred):
        y_pred = K.round(y_pred)
        return K.mean(K.maximum(K.batch_dot(y_pred, input_weights, 1) - 1, 0))

    return space_violation

In [9]:
model = get_model()

In [10]:
model.fit(x_train, y_train, epochs=512, verbose=2)

Train on 9000 samples
Epoch 1/512
9000/9000 - 0s - loss: 0.6496 - binary_accuracy: 0.6623 - space_violation: 1.0878 - overpricing: 0.4083
Epoch 2/512
9000/9000 - 0s - loss: 0.5653 - binary_accuracy: 0.7250 - space_violation: 0.4360 - overpricing: 0.2965
Epoch 3/512
9000/9000 - 0s - loss: 0.5200 - binary_accuracy: 0.7480 - space_violation: 0.3484 - overpricing: 0.2328
Epoch 4/512
9000/9000 - 0s - loss: 0.4809 - binary_accuracy: 0.7746 - space_violation: 0.2744 - overpricing: 0.2086
Epoch 5/512
9000/9000 - 0s - loss: 0.4436 - binary_accuracy: 0.8040 - space_violation: 0.2076 - overpricing: 0.1953
Epoch 6/512
9000/9000 - 0s - loss: 0.4105 - binary_accuracy: 0.8285 - space_violation: 0.1614 - overpricing: 0.1848
Epoch 7/512
9000/9000 - 0s - loss: 0.3839 - binary_accuracy: 0.8423 - space_violation: 0.1355 - overpricing: 0.1733
Epoch 8/512
9000/9000 - 0s - loss: 0.3631 - binary_accuracy: 0.8510 - space_violation: 0.1223 - overpricing: 0.1626
Epoch 9/512
9000/9000 - 0s - loss: 0.3473 - binary

In [11]:
model.evaluate(x_test,  y_test, verbose=2)
# It's the following:
# [loss, accuracy, space_violation, overpricing]

C:\Users\Guilherme\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


[0.24429048132896422, 0.8928, 0.035031185, 0.045922533]

In [12]:
# Understanding the data shape:

# x_train[0] are weights
# x_train[1] are prices

# model.predict([x_train[0], x_train[1]]) == model.predict(x_train)
print(model.predict([[x_train[0][0]], [x_train[1][0]]]))
print(np.array([x_train[0], x_train[1]]).shape)
print(np.array([[x_train[0][0]], [x_train[1][0]]]).shape)
# [
#  [
#   ... weights
#   ... 9000 entries
#  ]
#  [
#   ... prices
#   ... 9000 entries
#  ]
# ]

[[0.09659481 0.98779726 0.89629364 0.00433263 0.26424748]]
(2, 9000, 5)
(2, 1, 5)


In [13]:
print("5 first previsions: ")
for i in range(5):
  print(f"Features data - {i}")
  print("Weights:")
  print(x_train[0][i])
  print("Prices:")
  print(x_train[1][i])
  print("Actual output")
  print(y_train[i])
  print("Predicted output")
  predictions, = model.predict([[x_train[0][i]], [x_train[1][i]]])
  print(predictions)
  print("Predicted output rounded")
  print(np.round(predictions))
  print("\n")

5 first previsions: 
Features data - 0
Weights:
[1.13333333 0.26666667 0.03333333 1.43333333 0.5       ]
Prices:
[0.9787234  1.         0.15957447 0.75531915 0.12765957]
Actual output
[0. 1. 1. 0. 1.]
Predicted output
[0.09659481 0.98779726 0.89629364 0.00433263 0.26424748]
Predicted output rounded
[0. 1. 1. 0. 0.]


Features data - 1
Weights:
[1.46153846 1.92307692 1.61538462 2.84615385 2.69230769]
Prices:
[0.48387097 1.         0.53763441 0.84946237 0.15053763]
Actual output
[0. 0. 0. 0. 0.]
Predicted output
[1.1871278e-02 1.0654032e-02 4.5799911e-03 8.0499053e-04 9.8496348e-06]
Predicted output rounded
[0. 0. 0. 0. 0.]


Features data - 2
Weights:
[4.66666667 6.16666667 6.16666667 5.5        3.        ]
Prices:
[0.125    1.       0.28125  0.515625 0.09375 ]
Actual output
[0. 0. 0. 0. 0.]
Predicted output
[1.0511759e-04 4.1440129e-04 6.0831480e-05 1.7166436e-03 1.3001263e-03]
Predicted output rounded
[0. 0. 0. 0. 0.]


Features data - 3
Weights:
[1.59259259 1.37037037 0.74074074 0.96